In [75]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [76]:
import sys
root = '../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
#from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [77]:
def meanAveragePrecision(train_loader, test_loader, model, device='cuda') -> torch.Tensor:
    """
    Calculates mean average precision based on hashes from train_loader and test_loader.

    Args:
    ---
    train_loader : DataLoader
        DataLoader for training data.
    test_loader : DataLoader
        DataLoader for test data.
    model : torch.nn.Module
        Model used to compute hashes. Assumes model outputs a hash for each input.
    device : str
        The device on which to run computations ('cuda' or 'cpu').

    Returns:
    ---
    mean_average_precision : torch.Tensor
        The mean average precision value.
    """
    
    # Move model to the specified device
    model = model.to(device)
    model.eval()

    # Lists to store hashes and labels for training and testing data
    train_hashes, train_labels = [], []
    test_hashes, test_labels = [], []

    # Generate hashes and labels for the training set
    with torch.no_grad():  # Disable gradient computation for efficiency
        for images, labels, ind in tqdm(train_loader, desc="Processing Train Loader"):
            images = images.to(device)
            labels = labels.to(device)
            # Get the hash from the model (assuming model outputs hash directly)
            hashes = model(images).sign()  # Apply sign if needed for binary hash
            train_hashes.append(hashes)
            train_labels.append(labels)

        # Concatenate all hashes and labels along the batch dimension
        train_hashes = torch.cat(train_hashes, dim=0)
        train_labels = torch.cat(train_labels, dim=0)

    # Generate hashes and labels for the test set
    with torch.no_grad():
        for images, labels, ind in tqdm(test_loader, desc="Processing Test Loader"):
            images = images.to(device)
            labels = labels.to(device)
            hashes = model(images).sign()
            test_hashes.append(hashes)
            test_labels.append(labels)

        test_hashes = torch.cat(test_hashes, dim=0)
        test_labels = torch.cat(test_labels, dim=0)

    # Now calculate mean average precision
    aps = []
    for i in tqdm(range(test_hashes.size(0)), desc="Calculating Mean Average Precision"):
        test_hash = test_hashes[i]  # Get a single test hash
        label = test_labels[i]

        # Calculate distances between the single test hash and all train hashes
        distances = (train_hashes - test_hash).abs().sum(dim=1)

        # Create true positive indicators
        tp = (train_labels == label).float()

        # Sort by distances (ascending order)
        sorted_indices = distances.argsort()
        sorted_tp = tp[sorted_indices]

        # Cumulative sum of true positives
        cumsum_tp = torch.cumsum(sorted_tp, dim=0)

        # Precision calculation
        precision = cumsum_tp / (torch.arange(1, len(cumsum_tp) + 1, device=device).float())

        # Average precision calculation
        if sorted_tp.sum() > 0:  # Avoid division by zero
            ap = (precision * sorted_tp).sum() / sorted_tp.sum()
        else:
            ap = torch.tensor(0.0, device=device)
        aps.append(ap)
    
    # Calculate mean average precision
    mean_average_precision = torch.stack(aps).mean()
    
    return mean_average_precision

In [78]:
X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
X_train_tensor = torch.tensor(X_train)
y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)


X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
X_test_tensor = torch.tensor(X_test)
y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)


In [79]:
def CreateDataset(root, num_classes, batch_size, train = True):
    if train == True:
        #Create X_train_tensor
        X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
        X_train_tensor = torch.tensor(X_train)

        #Create Y_train_tensor
        y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_train_tensor = torch.nn.functional.one_hot(y_train_tensor, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)

        #Create indices
        indices_train = torch.arange(len(X_train_tensor))

        dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return train_loader

    else:
        X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
        X_test_tensor = torch.tensor(X_test)

        y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)
        y_test_tensor = torch.nn.functional.one_hot(y_test_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

        #Create indices
        indices_test = torch.arange(len(X_test_tensor))

        dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return test_loader

    #Missing implementation for Test and Validation
    

In [80]:
train_loader = CreateDataset(root, num_classes = 10, batch_size = 128)
test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = False)


In [81]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, 48),    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights based on paper
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases based on paper

    def forward(self, x):
        return self.fc_layers(x)

In [82]:
model = CustomNN().to(device)

In [83]:
class DPSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DPSHLoss, self).__init__()
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        s = (y @ self.Y.t() > 0).float()
        inner_product = u @ self.U.t() * 0.5

        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product

        likelihood_loss = likelihood_loss.mean()

        quantization_loss = eta * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

In [84]:
def train_val(device, train_loader, test_loader, train_size, test_size, batch_size, n_classes, bit, num_epoch, lr):


    optimizer = optim.Adam(model.parameters(), lr=0.001)

    criterion = DPSHLoss(train_size, n_classes, bit)


    #model.train()

    #Best_mAP = 0

    for epoch in range(num_epoch):

        current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))

        print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            "DPSH", epoch + 1, num_epoch, current_time, bit, "CIFAR"), end="")

        train_loss = 0
        for image, label, ind in train_loader:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            u = model(image)


            loss = criterion(u, label.float(), ind, lr)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / (train_size / batch_size)

        print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))

        if (epoch + 1) % 3 == 0:
            map = meanAveragePrecision(train_loader, test_loader, model, device)
            print(map)
            #Best_mAP = validate(config, Best_mAP, test_loader, dataset_loader, net, bit, epoch, num_dataset)


In [85]:
train_val(device, train_loader, test_loader, train_size = 45000, test_size = 10000, batch_size = 128, n_classes = 10, bit = 48, num_epoch = 150, lr = 0.1)

DPSH[ 1/150][13:15:43] bit:48, dataset:CIFAR, training... loss:0.79593
DPSH[ 2/150][13:15:45] bit:48, dataset:CIFAR, training... loss:0.78111
DPSH[ 3/150][13:15:48] bit:48, dataset:CIFAR, training... loss:0.78417


Calculating Mean Average Precision:   0%|          | 0/10000 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (10) must match the size of tensor b (45000) at non-singleton dimension 1